### Importing Important Libraries

#### Steps To Be Followed
1. Importing necessary Libraries
2. Creating S3 bucket 
3. Mapping train And Test Data in S3
4. Mapping The path of the models in S3

In [108]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session

In [109]:
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.inputs import TrainingInput

In [110]:
bucket_name = 'riskclassifier' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
my_region = boto3.session.Session().region_name # set the region of the instance
print(my_region)

us-east-2


In [111]:
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-2':
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': 'us-east-2'})
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [112]:
# set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://riskclassifier/xgboost-as-a-built-in-algo/output


#### Downloading The Dataset And Storing in S3

In [113]:
traindata = pd.read_csv('s3://riskclassifier/train.csv')

In [114]:
testdata = pd.read_csv('s3://riskclassifier/test.csv')

In [115]:
train_data = traindata.iloc[:,2:]

In [116]:
train_data.head()

,age,marital_status,income,risk
0,34,1,28060.70,0
1,37,1,28009.34,0
2,29,1,27614.60,0
3,33,1,27287.18,0
4,39,1,26954.06,0


In [117]:
testdata.head()

,age,marital_status,income,risk
0,23,1,27850,0
1,20,1,29002,1
2,45,1,44590,1
3,23,1,35729,0
4,26,1,22586,1


In [118]:
### Saving Train And Test Into Buckets
## We start with Train Data
import os
pd.concat([train_data['risk'], train_data.drop(['risk'], 
                                                axis=1)], 
                                                axis=1).to_csv('train_processed.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train_processed.csv')).upload_file('train_processed.csv')

In [119]:
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [120]:
# Test Data Into Buckets
pd.concat([testdata['risk'], testdata.drop(['risk'], axis=1)], axis=1).to_csv('test_processed.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test_processed.csv')).upload_file('test_processed.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

### Building Models Xgboot- Inbuilt Algorithm

In [121]:
xgboost_container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name, "1.2-1")

In [122]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [123]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=xgboost_container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait=600)

In [125]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

2020-11-15 19:59:43 Starting - Starting the training job...
2020-11-15 19:59:45 Starting - Launching requested ML instances......
2020-11-15 20:00:51 Starting - Preparing the instances for training...
2020-11-15 20:01:32 Downloading - Downloading input data
2020-11-15 20:01:32 Training - Downloading the training image...
2020-11-15 20:02:09 Uploading - Uploading generated training model.INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single node training.
INFO:roo

### Deploy Machine Learning Model As Endpoints

In [126]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

---------------!

#### Prediction of the Test Data

In [127]:
from sagemaker.serializers import CSVSerializer
import numpy as np
test_data_array = testdata.drop(['risk'], axis=1).values #load the data into an array
#xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12,)


In [128]:
predictions_array

array([0.04536301, 0.04536301, 0.9542889 , 0.38787341, 0.04536301,
       0.08107375, 0.05486783, 0.03083932, 0.52256519, 0.03083932,
       0.52256519, 0.14694066])

In [129]:
cm = pd.crosstab(index=testdata['risk'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; 
fn = cm.iloc[1,0]; 
tp = cm.iloc[1,1]; 
fp = cm.iloc[0,1];
p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "Bad Risk", "Good Risk"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("Bad Risk", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Good Risk", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 66.7%

Predicted      Bad Risk       Good Risk
Observed
Bad Risk       56% (5)     0% (0)
Good Risk       44% (4)    100% (3) 



#### Deleting The Endpoints

In [131]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


ClientError: An error occurred (ValidationException) when calling the DeleteEndpoint operation: Could not find endpoint "arn:aws:sagemaker:us-east-2:812297953590:endpoint/sagemaker-xgboost-2020-11-15-20-03-59-940".